In [1]:
import pandas as pd

In [9]:
df = pd.read_csv('filtered_data.csv', index_col = 0)
df.head()

,Date,ASTS,ASTSW,NPAUU,INVZ,INVZW,CGROU,FOA,FOA/WS,RPLA/U,...,HYACU,ASLE,ASLEW,MNCLU,XL,GOEV,GOEVW,CLVR,CLVRW,SAMAU
1,2019-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-04 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2019-01-07 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
announcements_df = pd.read_csv('filtered_announced_data.csv', index_col = 0)
announcements_df.head()

,Ticker,Announced,Closed
0,ASTS,12/16/2020,4/6/2021
1,INVZ,12/11/2020,4/5/2021
2,FOA,10/13/2020,4/1/2021
3,PSFE,12/7/2020,3/30/2021
4,ARVL,11/18/2020,3/24/2021
